In [18]:
import pandas as pd


In [19]:
data = pd.read_csv('all_fight_results.csv')
# Reverse dataframe
print(data)

       Weight class          Fighter 1 Result          Fighter 2  \
0         Flyweight  Alexandre Pantoja    def        Kai Asakura   
1      Welterweight  Shavkat Rakhmonov    def  Ian Machado Garry   
2       Heavyweight         Ciryl Gane    def   Alexander Volkov   
3     Featherweight     Bryce Mitchell    def        Kron Gracie   
4     Featherweight        Choi Doo-ho    def      Nate Landwehr   
...             ...                ...    ...                ...   
7960            NaN     Gerard Gordeau    def       Kevin Rosier   
7961            NaN       Ken Shamrock    def      Patrick Smith   
7962            NaN       Royce Gracie    def      Art Jimmerson   
7963            NaN       Kevin Rosier    def       Zane Frazier   
7964            NaN     Gerard Gordeau    def         Teila Tuli   

                                          Method  Round  Time    #  
0        Technical Submission (rear-naked choke)    2.0  2:05  715  
1     Decision (unanimous) (48–47, 48–47, 48–

In [20]:
df = data.iloc[::-1].reset_index(drop=True)
print(df)

       Weight class          Fighter 1 Result          Fighter 2  \
0               NaN     Gerard Gordeau    def         Teila Tuli   
1               NaN       Kevin Rosier    def       Zane Frazier   
2               NaN       Royce Gracie    def      Art Jimmerson   
3               NaN       Ken Shamrock    def      Patrick Smith   
4               NaN     Gerard Gordeau    def       Kevin Rosier   
...             ...                ...    ...                ...   
7960  Featherweight        Choi Doo-ho    def      Nate Landwehr   
7961  Featherweight     Bryce Mitchell    def        Kron Gracie   
7962    Heavyweight         Ciryl Gane    def   Alexander Volkov   
7963   Welterweight  Shavkat Rakhmonov    def  Ian Machado Garry   
7964      Flyweight  Alexandre Pantoja    def        Kai Asakura   

                                          Method  Round  Time    #  
0                                TKO (head kick)    NaN  0:26    1  
1                                  TKO (punch

In [21]:
print(pd.unique(df['Result']))

['def' 'vs']


In [22]:
import importlib
import elo_calculator

In [23]:
importlib.reload(elo_calculator)
from elo_calculator import EloCalculator, ExpDecayK

elo_peak = {}
elo_cur = {}
elo_history = {}
record = {}
for index, row in df.iterrows():
    num = row['#']
    A = row['Fighter 1']
    B = row['Fighter 2']
    if(A not in elo_cur):
        elo_cur[A] = {'Elo': 1200} # League of Legends starts everyone at 1200 elo
        elo_history[A] = []
        record[A] = {"wins": 0, "losses": 0, "draws": 0}
    if(B not in elo_cur):
        elo_cur[B] = {'Elo': 1200}
        elo_history[B] = []
        record[B] = {"wins": 0, "losses": 0, "draws": 0}
    A_old_elo = elo_cur[A]['Elo']
    B_old_elo = elo_cur[B]['Elo']
    A_fights = record[A]["wins"] + record[A]["losses"] + record[A]["draws"]
    B_fights = record[B]["wins"] + record[B]["losses"] + record[B]["draws"]
    # Update record
    if(row['Result'] == 'def'):
        winner = 1
        record[A]["wins"] += 1
        record[B]["losses"] += 1
    else:
        winner = 0.5
        record[A]["draws"] += 1
        record[B]["draws"] += 1
    A_record = f"{record[A]['wins']}-{record[A]['losses']}-{record[A]['draws']}"
    B_record = f"{record[B]['wins']}-{record[B]['losses']}-{record[B]['draws']}"
    Ka, Kb = ExpDecayK(A_fights, B_fights)
    A_new_elo, B_new_elo = EloCalculator(A_old_elo, B_old_elo, winner, Ka, Kb)
    # Update elo
    elo_cur[A]['Elo'] = A_new_elo
    elo_cur[B]['Elo'] = B_new_elo
    # Add to elo history
    elo_history[A].append({"Event #": num, "Elo": A_new_elo, "Record": A_record})
    elo_history[B].append({"Event #": num, "Elo": B_new_elo, "Record": B_record})
    # Update peak elo
    if(A not in elo_peak or elo_peak[A]['Elo'] <= A_new_elo):
        elo_peak[A] = {"Event #": num, "Elo": A_new_elo}
    if(B not in elo_peak or elo_peak[B]['Elo'] <= B_new_elo):
        elo_peak[B] = {"Event #": num, "Elo": B_new_elo}


In [24]:
# Convert dictionaries to dataframes
df_elo_peak = pd.DataFrame.from_dict(elo_peak, orient='index')
df_elo_cur = pd.DataFrame.from_dict(elo_cur, orient='index')

# For elo_history, we make a dictionary of dataframes for each fighter
dict_elo_history = {
    fighter: pd.DataFrame(history)
    for fighter, history in elo_history.items()
}

In [25]:
# Converting to League ranks
    # Calcuate the current rank cutoff given the league distribution. E.g. if top x% of players are Challenger in league, set top x% of fighters to that rank
# League Season 14 rank % of player base and percentiles:
    # Challenger = 0.021%, 0.021%
    # Grandmaster = 0.051%, 0.072%
    # Master = 0.34%, 0.41%
    # Diamond = 2.3%, 2.7%
    # Emerald = 8.1%, 10.8%
    # Platinum = 12%, 23%
    # Gold = 18%, 41%
    # Silver = 22%, 63%
    # Bronze = 21%, 84%
    # Iron = 14%, 100%
import numpy as np
challenger = df_elo_cur['Elo'].quantile(1 - 0.00021)
grandmaster = df_elo_cur['Elo'].quantile(1- 0.00072)
master = df_elo_cur['Elo'].quantile(1 - 0.0041)
diamond = df_elo_cur['Elo'].quantile(1 - 0.027)
emerald = df_elo_cur['Elo'].quantile(1 - 0.108)
platinum = df_elo_cur['Elo'].quantile(1 - 0.23)
gold = df_elo_cur['Elo'].quantile(1 - 0.41)
silver = df_elo_cur['Elo'].quantile(1 - 0.63)
bronze = df_elo_cur['Elo'].quantile(1 - 0.84)
iron = df_elo_cur['Elo'].quantile(1 - 1)

cutoffs = {
    'Iron': iron,
    'Bronze': bronze,
    'Silver': silver,
    'Gold': gold,
    'Platinum': platinum,
    'Emerald': emerald,
    'Diamond': diamond,
    'Master': master,
    'Grandmaster': grandmaster,
    'Challenger': challenger
}
print(cutoffs)


{'Iron': np.float64(1023.2379526105263), 'Bronze': np.float64(1116.4090727515963), 'Silver': np.float64(1150.0), 'Gold': np.float64(1200.5101767698577), 'Platinum': np.float64(1260.4531599594402), 'Emerald': np.float64(1329.917503268367), 'Diamond': np.float64(1422.6188247059667), 'Master': np.float64(1517.1410517542738), 'Grandmaster': np.float64(1587.1285945815287), 'Challenger': np.float64(1614.2704340628113)}


In [26]:
def AssignRank(elo):
    largest_value = float('-inf')
    rank = None
    for key, value in cutoffs.items():
        if value <= elo and value > largest_value:
            largest_value = value
            rank = key
    return rank


# Calculate ranks for current elos
df_elo_cur['Rank'] = df_elo_cur.apply(lambda row: AssignRank(row['Elo']), axis=1)

# Calculate ranks for peak elos
df_elo_peak['Rank'] = df_elo_peak.apply(lambda row: AssignRank(row['Elo']), axis=1)


# Calculate ranks for historical elos
for fighter, value in elo_history.items():
    for event in value:
        event['Rank'] = AssignRank(event['Elo'])

In [27]:
# Export tables
df_elo_cur.to_csv('Current_Elos.csv', index=True)
df_elo_peak.to_csv('Peak_Elos.csv', index=True)
# elo_history is exported manually using VS Code
